In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

In [32]:
#load
data_raw_cost = pd.read_excel('dataset_cost_emea.xlsx', sheet_name='cost')

In [33]:
data_raw_cost.head()

,04 FISCAL MONTH,02 SUPER REGION,03 SUB SEGMENT,Contact Center OH Expense,CS HQ Owned and Allocated,Delivery,Delivery OH,GBU Owned and Allocated,Level 2 Support,Non-Phone Expense,Phone Support Expense,Supply Chain,Supply Chain OH
0,201611,EMEA,BPS,0.0,693664.238160,3.323405e+06,0.0,179463.740000,0.0,174111.340000,813451.070000,6.439089e+06,0.0
1,201611,EMEA,CPS,0.0,364780.164371,1.888109e+06,0.0,35326.430000,0.0,167815.500000,629873.292110,2.018374e+06,0.0
2,201611,EMEA,HPS,0.0,197914.713908,8.743848e+04,0.0,236469.985660,0.0,11510.932293,783365.805755,2.081316e+06,0.0
3,201611,EMEA,HPS Supplies,0.0,0.000000,1.182815e+04,0.0,39964.321996,0.0,0.000000,96773.087480,9.052318e+04,0.0
4,201611,EMEA,OPS,0.0,200157.794252,5.793384e+05,0.0,634820.669357,0.0,161122.480000,240401.428957,1.766240e+06,0.0


In [34]:
data_raw_cost = data_raw_cost.melt(id_vars=('04 FISCAL MONTH', '02 SUPER REGION', '03 SUB SEGMENT'), value_vars=('Contact Center OH Expense',
                                                                                            'CS HQ Owned and Allocated',
                                                                                            'Delivery',
                                                                                            'Delivery OH',
                                                                                            'GBU Owned and Allocated',
                                                                                            'Level 2 Support',
                                                                                            'Non-Phone Expense',
                                                                                            'Phone Support Expense',
                                                                                            'Supply Chain',
                                                                                            'Supply Chain OH'), var_name='line_cost', value_name='y')

data_raw_cost = data_raw_cost.groupby(by=['04 FISCAL MONTH', '02 SUPER REGION', '03 SUB SEGMENT', 'line_cost'], as_index=False)['y'].sum()

In [35]:
data_raw_reg = pd.read_excel('dataset_cost_emea.xlsx', sheet_name='reg')

In [36]:
data_raw_reg.head()

,04 FISCAL MONTH,02 SUPER REGION,03 SUB SEGMENT,Total Number of Calls,Total Number of Repair Events,WIB
0,201611,EMEA,BPS,124468.488370,66802.000000,33797240.0
1,201611,EMEA,CPS,66787.702776,34471.957197,8775266.0
2,201611,EMEA,HPS,75964.968572,25038.816923,9872760.0
3,201611,EMEA,HPS Supplies,7924.010000,4127.000000,116588789.0
4,201611,EMEA,OPS,27143.488059,9259.000000,1754763.0


In [37]:
all_data = pd.merge(data_raw_cost, data_raw_reg, how='right', on=['04 FISCAL MONTH', '02 SUPER REGION', '03 SUB SEGMENT'])
all_data.reset_index(inplace=True, drop=True)

In [38]:
all_data.head()

,04 FISCAL MONTH,02 SUPER REGION,03 SUB SEGMENT,line_cost,y,Total Number of Calls,Total Number of Repair Events,WIB
0,201611,EMEA,BPS,CS HQ Owned and Allocated,6.936642e+05,124468.48837,66802.0,33797240.0
1,201611,EMEA,BPS,Contact Center OH Expense,0.000000e+00,124468.48837,66802.0,33797240.0
2,201611,EMEA,BPS,Delivery,3.323405e+06,124468.48837,66802.0,33797240.0
3,201611,EMEA,BPS,Delivery OH,0.000000e+00,124468.48837,66802.0,33797240.0
4,201611,EMEA,BPS,GBU Owned and Allocated,1.794637e+05,124468.48837,66802.0,33797240.0


In [39]:
month_as_date_str_start = [str(x)[:4] + '-' + str(x)[4:] for x in all_data['04 FISCAL MONTH']]

In [40]:
month_date_start = pd.to_datetime(month_as_date_str_start)

In [41]:
month_as_date_str_end = [str(x.year) + '-' + str(x.month) + '-' + str(x.days_in_month) for x in month_date_start]

In [42]:
month_date_end = pd.to_datetime(month_as_date_str_end)

In [43]:
all_data['ds'] = month_date_end

In [44]:
all_data['unique_id'] = all_data['02 SUPER REGION'] + "_" + all_data['03 SUB SEGMENT'] + "_"+ all_data['line_cost']

In [45]:
Y_df = all_data[['ds',
                'unique_id',
                'y']]

X_df = all_data[['ds',
                'unique_id',
                'WIB',
                'Total Number of Calls',
                'Total Number of Repair Events']]

In [46]:
dates = month_date_end.unique()
dtrain = dates[:-3]
dtest = dates[-3:]

In [47]:
dtrain

DatetimeIndex(['2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28',
               '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30',
               '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31',
               '2017-11-30', '2017-12-31', '2018-01-31', '2018-02-28',
               '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30',
               '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31',
               '2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28',
               '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30',
               '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
               '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
               '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30',
               '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31',
               '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
               '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30',
      

In [48]:
dtest

DatetimeIndex(['2023-03-31', '2023-04-30', '2023-05-31'], dtype='datetime64[ns]', freq=None)

In [49]:
Y_train = Y_df.query('ds in @dtrain')
Y_test = Y_df.query('ds in @dtest')

X_train = X_df.query('ds in @dtrain') 
X_test = X_df.query('ds in @dtest')

In [50]:
len(Y_train.unique_id.unique()), len(Y_test.unique_id.unique())

(60, 60)

In [51]:
len(X_train.unique_id.unique()), len(X_test.unique_id.unique())

(60, 60)

In [52]:
train = Y_train.merge(X_df, how = 'left', on = ['unique_id', 'ds'])
train.head()

,ds,unique_id,y,WIB,Total Number of Calls,Total Number of Repair Events
0,2016-11-30,EMEA_BPS_CS HQ Owned and Allocated,6.936642e+05,33797240.0,124468.48837,66802.0
1,2016-11-30,EMEA_BPS_Contact Center OH Expense,0.000000e+00,33797240.0,124468.48837,66802.0
2,2016-11-30,EMEA_BPS_Delivery,3.323405e+06,33797240.0,124468.48837,66802.0
3,2016-11-30,EMEA_BPS_Delivery OH,0.000000e+00,33797240.0,124468.48837,66802.0
4,2016-11-30,EMEA_BPS_GBU Owned and Allocated,1.794637e+05,33797240.0,124468.48837,66802.0


In [53]:
train.tail()

,ds,unique_id,y,WIB,Total Number of Calls,Total Number of Repair Events
4555,2023-02-28,EMEA_OPS Supplies_Level 2 Support,0.00,11158695.0,2.0,2.0
4556,2023-02-28,EMEA_OPS Supplies_Non-Phone Expense,1072.97,11158695.0,2.0,2.0
4557,2023-02-28,EMEA_OPS Supplies_Phone Support Expense,572.16,11158695.0,2.0,2.0
4558,2023-02-28,EMEA_OPS Supplies_Supply Chain,1162.99,11158695.0,2.0,2.0
4559,2023-02-28,EMEA_OPS Supplies_Supply Chain OH,1707.72,11158695.0,2.0,2.0


In [54]:
X_test.head()

,ds,unique_id,WIB,Total Number of Calls,Total Number of Repair Events
4560,2023-03-31,EMEA_BPS_CS HQ Owned and Allocated,43396749.0,52452.0,53590.0
4561,2023-03-31,EMEA_BPS_Contact Center OH Expense,43396749.0,52452.0,53590.0
4562,2023-03-31,EMEA_BPS_Delivery,43396749.0,52452.0,53590.0
4563,2023-03-31,EMEA_BPS_Delivery OH,43396749.0,52452.0,53590.0
4564,2023-03-31,EMEA_BPS_GBU Owned and Allocated,43396749.0,52452.0,53590.0


In [100]:
from statsforecast.models import AutoARIMA, AutoETS, AutoTheta, AutoCES, SimpleExponentialSmoothingOptimized, SeasonalExponentialSmoothingOptimized, SimpleExponentialSmoothing, WindowAverage

In [102]:
season_length = 12
models = [
    AutoARIMA(season_length=season_length), # ARIMA model with automatic order selection and seasonal component
    SimpleExponentialSmoothingOptimized(),
    SeasonalExponentialSmoothingOptimized(season_length=season_length)
    #AutoETS(season_length=season_length) # ETS model with automatic error, trend, and seasonal component
    #AutoTheta(season_length=season_length) # Theta model with automatic seasonality detection
]

models = [
    SimpleExponentialSmoothingOptimized(), # ARIMA model with automatic order selection and seasonal component
    SimpleExponentialSmoothing(0.9),
    WindowAverage(window_size=6)
]

sf = StatsForecast(
    models=models,
    freq='M',
    n_jobs=-1
)

In [103]:
horizon = 3
level = [95]

fcst = sf.forecast(df=train, h=horizon, X_df=X_test)
fcst = fcst.reset_index()

In [104]:
StatsForecast.plot(Y_df, fcst, max_insample_length=3)

In [105]:
res = Y_test.merge(fcst, how='left', on=['unique_id', 'ds'])
mae_arima = abs(res['y']-res['AutoARIMA']).mean()
print('The MAE with exogenous regressors is '+str(round(mae_arima,2)))

KeyError: 'AutoARIMA'

In [95]:
mae_ses = abs(res['y']-res['SESOpt']).mean()
print('The MAE with exogenous regressors is '+str(round(mae_ses,2)))

The MAE with exogenous regressors is 90452.68


In [97]:
mae_sso = abs(res['y']-res['SES']).mean()
print('The MAE with exogenous regressors is '+str(round(mae_sso,2)))

The MAE with exogenous regressors is 96153.3


In [106]:
res[res['unique_id'] == 'EMEA_CPS_Supply Chain']

,ds,unique_id,y,SESOpt,SES,WindowAverage
18,2023-03-31,EMEA_CPS_Supply Chain,1655709.23,1700103.625,1569941.25,1892970.375
78,2023-04-30,EMEA_CPS_Supply Chain,1561130.16,1700103.625,1569941.25,1892970.375
138,2023-05-31,EMEA_CPS_Supply Chain,1523408.79,1700103.625,1569941.25,1892970.375


In [62]:
mae = abs(res['y']-res['AutoARIMA']).mean()
print('The MAE with exogenous regressors is '+str(round(mae,2)))

The MAE with exogenous regressors is 125200.02


In [63]:
mae = abs(res[res['ds'] == '2023-03-31']['y'] - res[res['ds'] == '2023-03-31']['AutoARIMA']).mean()

In [64]:
print('The MAE with exogenous regressors is '+str(round(mae,2)))

The MAE with exogenous regressors is 142448.57
